## DV1597 Assignment 1

Namn

***

#### Initialize

In [72]:
# Import packages
import pandas as pd
import matplotlib.pyplot as mpl
import seaborn as sb
import numpy as np
import math

In [73]:
# Load dataset
df_original = pd.read_csv('SMHI_Dataset/SMHI_Dataset/SMHI_Data.csv')
df = df_original.copy()
df

,Datum,Tid (UTC),Lufttemperatur
0,2010-03-01,00:00:00,2.0
1,2010-03-01,01:00:00,2.2
2,2010-03-01,02:00:00,2.2
3,2010-03-01,03:00:00,1.7
4,2010-03-01,04:00:00,1.7
...,...,...,...
122851,2024-01-01,02:00:00,3.9
122852,2024-01-01,03:00:00,3.9
122853,2024-01-01,04:00:00,3.7
122854,2024-01-01,05:00:00,3.6


***

### Part 1

#### Q1
**Explain the data cleaning steps you consider for analyzing the provided dataset. You must motivate and describe why, how, and in which order you are going to apply the selected steps to the dataset. Note that you should consider applying your proposed data cleaning steps on the dataset to answer Q2-Q11.**

Firstly we will look at the dataset and remove all duplicates in it. Then we want to remove all the missing values. Some missing values have an entry in the csv-file, but is missing the temperature data, while some timestamps are not found at all in the data. The strategy here is to insert all timestamps that are missing, with no temperature data, and then add an estimated temperature for every missing instance.

#### Q3
**Which strategy was used for handling the missing values? Motivate why you consider the selected strategy the most suitable for this task.**

#### Q5
**Plot all available air temperature data as a line plot with dates on the x-axis.**

#### Q6
**Rank the overall temperature per month, sort from the coldest to the warmest. List the top ten (10) coldest months in the dataset. Format the output as: YEAR/Month: temp, e.g.“2010/January: -5.0”**

***

#### Q2
**Does the dataset contain any missing values? If so, how many in both absolute terms and percentages? Reflect on the number of missing values. Do you regard it as much or not?**

### Find missing timestamps

Out first step is to look at every row in the dataset, and find if any timestamp is missing from it. For example if for a date, the time 00:00:00 and 02:00:00 exists, but not 01:00:00. We also add all the timestamps that are missing to the dataframe.

In [74]:
# # Missing values investigation
df = df.drop_duplicates().reset_index(drop=True)
date = None

missing_times = []
timestamps = []

for i, row in df.iterrows(): # Iterate over all rows
    if date is None:
        date = row['Datum']
    time = int(row['Tid (UTC)'][:2]) # Get time from data point
    
    if row['Datum'] != date: # New date found
        for ii in range(24): # Count if all times are included
            counter = timestamps.count(ii)
            if counter == 0: # Time not found, add to missing_times
                missing_times.append(date + ","+ str(ii))
        date = row['Datum']
        time = int(row['Tid (UTC)'][:2])
        timestamps = [time]
    else:
        timestamps.append(time)

print("Missing timestamps:", len(missing_times))

for m in missing_times: # Add missing timestamps to dataframe
    date, time = m.split(',')
    timestring = ""
    if len(time) < 2:
        timestring += "0"+time
    else:
        timestring += time
    timestring+=":00:00"
    df.loc[len(df)] = [date,timestring,None]

df.sort_values(by=['Datum','Tid (UTC)'], inplace=True)
df = df.reset_index(drop=True)

Missing timestamps: 694


We found and added a total of 694 missing timestamps.

### Missing temperature values
Now we look for instances where the temperature data is missing.

In [75]:
na_values = df.isna().sum() # Count NaN values for each column
na_temp = na_values['Lufttemperatur']
na_p = round(100*(na_temp/len(df)),2)

print("NaN temperatures:",na_temp)
print("NaN temperatures percentage:",na_p, '%')

NaN temperatures: 7931
NaN temperatures percentage: 6.54 %


### Estimating missing values

In [76]:
def estimate(index):
    i = index - 24
    before = []
    after = []
    while i > 0 and len(before) < 3:
        temp = df.iloc[i]['Lufttemperatur']
        if not math.isnan(temp):
            before.append(temp)
        i-=24
        
    i = index + 24
    while i < len(df) and len(after) < 3:
        temp = df.iloc[i]['Lufttemperatur']
        if not math.isnan(temp):
            after.append(temp)
        i+=24
        
    all_temps = []
    all_temps.extend(before)
    all_temps.extend(after)
    
    mean = round(sum(all_temps)/len(all_temps),1)
    return mean

for i, row in df.iterrows():
    if math.isnan(row['Lufttemperatur']):
        date = row['Datum']
        before = df.iloc[i-1]['Lufttemperatur']
        after = df.iloc[i+1]['Lufttemperatur']
        
        if math.isnan(after):
            after = estimate(i)
        mean = round((before+after)/2,1)
        df.loc[i,'Lufttemperatur'] = mean
        
df.to_csv('data.csv', index=False)

***
#### Q4

**Calculate the following statistics for the air temperature values in the dataset: minimum value, maximum value, sample mean, Q1/Q2/Q3 quartiles, sample standard deviation, 95% confidence interval of the mean**

In [96]:
temps = df_original['Lufttemperatur']
print("Min:",temps.min())
print("Mean:",temps.max())
print("Mean:",round(temps.mean(),3))

print("\nQuartiles:")
print("Q1 (25%):",temps.quantile(0.25))
print("Q2 (50%):",temps.quantile(0.5))
print("Q3 (75%):",temps.quantile(0.75))

print("\nStandard deviation",round(temps.std(),3))


Min: -19.2
Mean: 38.6
Mean: 9.435

Quartiles:
Q1 (25%): 4.1
Q2 (50%): 9.1
Q3 (75%): 15.3

Standard deviation 6.978


### Part 2 (D)

Q7. Which distribution is suitable to consider to fit the air temperature data? How well does the data follow that distribution given some measure, e.g., p-value?

***

### Part 3 (C)

Q8. Do you regard any of the air temperature measures in the dataset to be extreme values/outliers? If so, how many values? Motivate why you
consider these values as anomalies.

Q9. Calculate the mean temperature for each day in the year 2023 and then plot those means using a line plot with dates on the x-axis.

***

### Part 4 (B)

Q10. Rank the overall temperature per two (2) consecutive months starting with the warmest 2-month period first, e.g., (Y1M1+Y1M2)/2, (Y1M2+Y1M3)/2,... (Y1M11+Y1M1)/2, (Y1M12+Y2M1)/2,.... After that, you should sort those means from highest to lowest and list the ten (10) warmest 2-month periods all together in the dataset.

***

### Part 5 (A)

Q11. Calculate the 95% confidence intervals (CI) for the mean of each daily average in Q9 and add it to the line plot that shows the mean values of
each day of the year 2023. You can, for example, solve this by plotting the upper and lower bounds as lines above and below the mean line in your
plot. Another more nicely looking solution is to visualize the CI as shades around the mean line in your plot.